## Recursive inference (multi-step) for BSCTRFM models

## this script:
### uses time series instead of SLDB arrays for easier and more efficient timestamp management
### uses raw data (no outlier removal)
### trims time series with zeros at the beginning of the training dataset

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [2]:
# tf.random.set_seed(1234)

In [3]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [4]:
# converts a set of tensors to a feature dict to a serialized example to pass it
# to the prediction function of the saved model 
def input_tensors_to_serialized_example(encoder_input_float_tensor,
                                        decoder_input_float_tensor,
                                        id_float_tensor):
    # first, pass the float tensors to NumPy array, then flatten them
    encoder_input_flat_array = encoder_input_float_tensor.numpy().flatten()
    decoder_input_flat_array = decoder_input_float_tensor.numpy().flatten()
    id_flat_array = id_float_tensor.numpy().flatten()
    
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # feature definition
            feature={
                'encoder_input': _float_feature_from_list_of_values(encoder_input_flat_array),
                'decoder_input': _float_feature_from_list_of_values(decoder_input_flat_array),
                'id': _float_feature_from_list_of_values(id_flat_array)
            }
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    serialized_example = tf.convert_to_tensor(numpy_example)

    return serialized_example

In [5]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [6]:
# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [7]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [8]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [9]:
# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR TRANSFORMER-BASED MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY TO PRODUCE DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '{:03d}'.format(forecast_window)

In [10]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [11]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [12]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']
ts_identifier

'LD2011-2014_SEPARATED_FULL'

In [13]:
m = 168
m

168

In [14]:
t = 168
t

168

In [29]:
model_ids = [
    'BSCTRFM_TPU_064',
    'BSCTRFM_TPU_064',
    'BSCTRFM_TPU_064',
    'BSCTRFM_TPU_064',
    'BSCTRFM_TPU_064',
    'BSCTRFM_TPU_064',
    'BSCTRFM_TPU_064',
    'BSCTRFM_TPU_064',
  ]

In [30]:
executions = [
    2, 3, 4, 5, 6, 7, 8, 9,
]

In [31]:
saved_model_ids = [
    1648836416,
    1648836709,
    1648837002,
    1648837294,
    1648837593,
    1648837890,
    1648838192,
    1648838498,
]

In [32]:
for model_id, execution, saved_model_id in zip(model_ids,
                                               executions,
                                               saved_model_ids):
    print(model_id, execution, saved_model_id)

BSCTRFM_TPU_064 2 1648836416
BSCTRFM_TPU_064 3 1648836709
BSCTRFM_TPU_064 4 1648837002
BSCTRFM_TPU_064 5 1648837294
BSCTRFM_TPU_064 6 1648837593
BSCTRFM_TPU_064 7 1648837890
BSCTRFM_TPU_064 8 1648838192
BSCTRFM_TPU_064 9 1648838498


In [33]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
]

In [34]:
decoder_input_columns = encoder_input_columns

In [35]:
id_columns = ['token_id']

In [36]:
# use a dictionary to remain the code consistent with the SLDB building process
# most of the times, only ts['test'] will be used for inference
# however, ts['eval'] might also be used, as it have not really been seen by training process
# (no modification in model training resulted from evaluation stage)

### rename the time series dictionary to ts_test, use the customer_id as key

In [37]:
start, end = 1, 370

In [38]:
customer_ids = ['MT_{:03d}'.format(token_id) for token_id in np.arange(start, end + 1)]

In [39]:
len(customer_ids)

370

In [40]:
# test time series and scalers are loaded just once, and kept in memory to speed up inference

ts_test = dict()
scaler = dict()

for customer_id in customer_ids:

    # read the time series for the current customer
    ts_test[customer_id] = pd.read_pickle('{}/test/{}.pkl'.format(data_dir, customer_id))

    # for consistency, rename the column 'date' as 'timestamp'
    ts_test[customer_id] = ts_test[customer_id].rename(columns={"date": "timestamp"})
    # set the column 'timestamp' as index
    ts_test[customer_id] = ts_test[customer_id].set_index('timestamp')

    # report start and end timestamp for the loaded time series
    # print('Loaded test time series for {}, which spans from {} to {}'.\
    #      format(customer_id,
    #             str(ts_test[customer_id].index[:1][0]),
    #             str(ts_test[customer_id].index[-1:][0])))
    
    # build a path to the scaler of the time series
    scaler_path = '{}/scalers/min_max_{}.save'.format(data_dir, customer_id)
    # and load it
    scaler[customer_id] = joblib.load(scaler_path)
    # print('Loaded scaler on test time series for {}'.format(customer_id))



## skip the following code to avoid inference process

In [41]:
# recursive inference process iterate on:
#    model_id/execution/saved_model_id combinations
#    customer_id
#    dataset_row_indexes (0, 24, 48, 72, 96, 120, 144)
#    forecast_window

In [42]:
# a second inference identifier to run more than one inference
# on the same combination model_id, execution, inference;
# to produce different inference processes because predict_fn is stochastic
for event in [0]:

    for model_id, execution, saved_model_id in zip(model_ids,
                                                   executions,
                                                   saved_model_ids):

        # use model identifier and execution number to build the model directory string
        model_dir = '{}_{:02d}'.format(model_id, execution)

        # get the path to the saved model main directory
        saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                             'models',
                                                             model_dir)

        # get all the files in the saved model path, to find the most recent one
        # all_files = os.listdir(saved_model_path)
        # get the path to the most recent saved model
        # latest_saved_model_id = sorted(all_files)[-1]

        # build the full path for the latest saved model dir
        export_dir = '{}/{}'.format(saved_model_path, saved_model_id)
        print ('Exported model path is {}'.format(export_dir))

        # load the saved model and the prediction function
        imported = load(export_dir=export_dir, tags='serve')
        predict_fn = imported.signatures["serving_default"]

        # a dictionary to store predictions detail dataframe per station id
        # predictions_detail = dict()

        # a columns list for the predictions dataframe
        pred_df_columns = [
            'model_id',
            'execution',
            'saved_model_id',
            'dataset',
            'inference',
            'event',
            'customer_id',
            'timestamp',
            'prediction',
            'target'
        ]

        # a dataframe for global inferences
        global_inferences_df = pd.DataFrame(columns=pred_df_columns)

        for customer_id in customer_ids:

            # build the predictions dataframe as a key-value pair of the dictionary
            # predictions_detail[customer_id] = pd.DataFrame(columns=pred_df_columns)

            # iterate on a set of valid rows of the test dataset
            # starting_point = 0 # based on the inference dataset
            # span = 1 + 6*24 # number of days in the test dataset, expressed in hours

            # MAKE THE PREDICTION SETS FOR BENCHMARKING ONLY! 
            # dataset_row_indexes_list = starting_point + np.arange(span)
            dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]

            for start_index in dataset_row_indexes_list:

                # define first prediction interval with start- and end-index
                # given the interval time_series[start_index:end_index]
                # the conditioning range is the union of the encoder-input and the decoder-input
                # and the prediction range is only the last lecture in the interval,
                # by means of a recursive inference process
                # on each step the last prediction is added to the decoder input
                # and the prediction range grows one step into the future

                # get the end-index of this recursive inference interval
                end_index = start_index + (m + t)

                # initialize a list to store recurrent predictions for this interval
                predictions_list = list()

                for i in np.arange(forecast_window):

                    # build the inference interval as a sub-series of the dataset
                    # the following slicing operation is equivalent to:
                    # sub_series = ts_test[customer_id].iloc[start_index + i : end_index + i]
                    sub_series = ts_test[customer_id][start_index + i : end_index + i]

                    # important: build sources as copies of the sub-series (and therefore of the global time series)
                    # to avoid overwriting the original dataset

                    # the encoder input source
                    encoder_input = sub_series[encoder_input_columns][:m].copy()

                    # the decoder input source
                    # decoder_input = sub_series[m-1:-1].copy()
                    decoder_input = sub_series[decoder_input_columns][m-1:m-1+t].copy()

                    # the id (integer) for the customer
                    id_input = sub_series[id_columns][:1].copy()

                    # on first step (i=0), the decoder input carries only true values
                    # and the predictions list is empty
                    # on subsequent steps, the decoder input includes all previous predictions
                    # (stored in the predictions list)
                    if i > 0:
                        decoder_input['kw_scaled'][-i:] = predictions_list

                    # the target source, for metrics calculation
                    # the first part of the sub-series is the encoder input, and
                    # the second part of the sub-series is the target (only the variable column!)
                    target = sub_series['kw_scaled'][m:].copy()

                    # build source tensors from the sub-series    
                    encoder_input_tensor = tf.expand_dims(encoder_input, axis=0)
                    decoder_input_tensor = tf.expand_dims(decoder_input, axis=0)
                    id_tensor = tf.expand_dims(id_input, axis=0)

                    # make input example for the prediction function
                    input_example = input_tensors_to_serialized_example(encoder_input_tensor,
                                                                        decoder_input_tensor,
                                                                        id_tensor)

                    # get the output of the prediction function as a dictionary
                    predict_output_dict = predict_fn(input_example)

                    # get the prediction output tensor
                    predict_output_tensor = predict_output_dict['forecast']

                    # get the most recent prediction
                    most_recent_prediction = predict_output_tensor[0, :, 0].numpy()[-1]

                    # append the most recent prediction timestep to the predictions list
                    predictions_list.append(most_recent_prediction)

                # iterative predictions over the forecast window reside in predictions_list
                # convert list to array, then expand feature dimension with value 1
                predicted_values = np.array(predictions_list).reshape(-1, 1)

                # inverse-scale predictions
                rescaled_predicted_values = scaler[customer_id].inverse_transform(predicted_values)

                # and the true values remain in the prediction tensor, pass them to a NumPy array
                # for the true values array, expand feature dimension with value 1
                true_values = np.array(target[-i-1:]).reshape(-1, 1)

                # inverse-scale true values
                rescaled_true_values = scaler[customer_id].inverse_transform(true_values)

                # a temporary dataframe built from the data in the current row
                df = pd.DataFrame(columns=pred_df_columns)
                df['model_id'] = forecast_window*[model_id]
                df['execution'] = forecast_window*[execution]
                df['saved_model_id'] = forecast_window*[saved_model_id]
                df['dataset'] = forecast_window*[dataset]
                df['inference'] = forecast_window*[inference]
                df['event'] = forecast_window*[event]
                df['customer_id'] = forecast_window*[customer_id]
                df['timestamp'] = target.index[-i-1:]
                df['prediction'] = np.squeeze(rescaled_predicted_values)
                df['target'] = np.squeeze(rescaled_true_values)

                # append the temporary dataframe to the predictions detail dataframe
                # predictions_detail[customer_id] = pd.concat([predictions_detail[customer_id], df])
                global_inferences_df = pd.concat([global_inferences_df, df])

            print('Finished inferences for {}'.format(customer_id))

            # reset the index of final dataframe, once all of its rows (dataset) have been processed
            # predictions_detail[customer_id] = predictions_detail[customer_id].reset_index(drop=True)

            # do not persist a single pickle file per customer_id
            persist_separated_pickles = False

            if persist_separated_pickles:

                # build a path to persist the dataframe to database
                detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}_{}.pkl'.format(
                    PROJECT_ROOT,
                    'inferences',
                    'electricity',
                    model_id,
                    execution,
                    saved_model_id,
                    dataset,
                    inference,
                    event,
                    customer_id)

                # persist the Pandas dataframe to database
                predictions_detail[customer_id].to_pickle(detail_pickle_path)
                print(
                    'Persisted predictions detail of {}_{:02d}_{}_{}_{}_{:02d}_{}'.format(model_id,
                                                                                       execution,
                                                                                       saved_model_id,
                                                                                       dataset,
                                                                                       inference,
                                                                                       event,
                                                                                       customer_id)
                )


        # reset index for global inferences dataframe
        global_inferences_df = global_inferences_df.reset_index(drop=True)

        # build a path to persist the global inferences dataframe
        global_inferences_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
            PROJECT_ROOT,
            'inferences',
            'electricity',
            model_id,
            execution,
            saved_model_id,
            dataset,
            inference,
            event
        )

        # persist the global inference dataframe
        global_inferences_df.to_pickle(global_inferences_pickle_path)
        print('Persisted global inferences to {}'.format(global_inferences_pickle_path))


Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_064_02/export/exporter/1648836416
Finished inferences for MT_001
Finished inferences for MT_002
Finished inferences for MT_003
Finished inferences for MT_004
Finished inferences for MT_005
Finished inferences for MT_006
Finished inferences for MT_007
Finished inferences for MT_008
Finished inferences for MT_009
Finished inferences for MT_010
Finished inferences for MT_011
Finished inferences for MT_012
Finished inferences for MT_013
Finished inferences for MT_014
Finished inferences for MT_015
Finished inferences for MT_016
Finished inferences for MT_017
Finished inferences for MT_018
Finished inferences for MT_019
Finished inferences for MT_020
Finished inferences for MT_021
Finished inferences for MT_022
Finished inferences for MT_023
Finished inferences for MT_024
Finished inferences for MT_025
Finished inferences for MT_026
Finished inferences for MT_027
Finished inferences for MT_028
Finished inferences for MT

Finished inferences for MT_262
Finished inferences for MT_263
Finished inferences for MT_264
Finished inferences for MT_265
Finished inferences for MT_266
Finished inferences for MT_267
Finished inferences for MT_268
Finished inferences for MT_269
Finished inferences for MT_270
Finished inferences for MT_271
Finished inferences for MT_272
Finished inferences for MT_273
Finished inferences for MT_274
Finished inferences for MT_275
Finished inferences for MT_276
Finished inferences for MT_277
Finished inferences for MT_278
Finished inferences for MT_279
Finished inferences for MT_280
Finished inferences for MT_281
Finished inferences for MT_282
Finished inferences for MT_283
Finished inferences for MT_284
Finished inferences for MT_285
Finished inferences for MT_286
Finished inferences for MT_287
Finished inferences for MT_288
Finished inferences for MT_289
Finished inferences for MT_290
Finished inferences for MT_291
Finished inferences for MT_292
Finished inferences for MT_293
Finished

Finished inferences for MT_149
Finished inferences for MT_150
Finished inferences for MT_151
Finished inferences for MT_152
Finished inferences for MT_153
Finished inferences for MT_154
Finished inferences for MT_155
Finished inferences for MT_156
Finished inferences for MT_157
Finished inferences for MT_158
Finished inferences for MT_159
Finished inferences for MT_160
Finished inferences for MT_161
Finished inferences for MT_162
Finished inferences for MT_163
Finished inferences for MT_164
Finished inferences for MT_165
Finished inferences for MT_166
Finished inferences for MT_167
Finished inferences for MT_168
Finished inferences for MT_169
Finished inferences for MT_170
Finished inferences for MT_171
Finished inferences for MT_172
Finished inferences for MT_173
Finished inferences for MT_174
Finished inferences for MT_175
Finished inferences for MT_176
Finished inferences for MT_177
Finished inferences for MT_178
Finished inferences for MT_179
Finished inferences for MT_180
Finished

Finished inferences for MT_036
Finished inferences for MT_037
Finished inferences for MT_038
Finished inferences for MT_039
Finished inferences for MT_040
Finished inferences for MT_041
Finished inferences for MT_042
Finished inferences for MT_043
Finished inferences for MT_044
Finished inferences for MT_045
Finished inferences for MT_046
Finished inferences for MT_047
Finished inferences for MT_048
Finished inferences for MT_049
Finished inferences for MT_050
Finished inferences for MT_051
Finished inferences for MT_052
Finished inferences for MT_053
Finished inferences for MT_054
Finished inferences for MT_055
Finished inferences for MT_056
Finished inferences for MT_057
Finished inferences for MT_058
Finished inferences for MT_059
Finished inferences for MT_060
Finished inferences for MT_061
Finished inferences for MT_062
Finished inferences for MT_063
Finished inferences for MT_064
Finished inferences for MT_065
Finished inferences for MT_066
Finished inferences for MT_067
Finished

Finished inferences for MT_301
Finished inferences for MT_302
Finished inferences for MT_303
Finished inferences for MT_304
Finished inferences for MT_305
Finished inferences for MT_306
Finished inferences for MT_307
Finished inferences for MT_308
Finished inferences for MT_309
Finished inferences for MT_310
Finished inferences for MT_311
Finished inferences for MT_312
Finished inferences for MT_313
Finished inferences for MT_314
Finished inferences for MT_315
Finished inferences for MT_316
Finished inferences for MT_317
Finished inferences for MT_318
Finished inferences for MT_319
Finished inferences for MT_320
Finished inferences for MT_321
Finished inferences for MT_322
Finished inferences for MT_323
Finished inferences for MT_324
Finished inferences for MT_325
Finished inferences for MT_326
Finished inferences for MT_327
Finished inferences for MT_328
Finished inferences for MT_329
Finished inferences for MT_330
Finished inferences for MT_331
Finished inferences for MT_332
Finished

...

Finished inferences for MT_340
Finished inferences for MT_341
Finished inferences for MT_342
Finished inferences for MT_343
Finished inferences for MT_344
Finished inferences for MT_345
Finished inferences for MT_346
Finished inferences for MT_347
Finished inferences for MT_348
Finished inferences for MT_349
Finished inferences for MT_350
Finished inferences for MT_351
Finished inferences for MT_352
Finished inferences for MT_353
Finished inferences for MT_354
Finished inferences for MT_355
Finished inferences for MT_356
Finished inferences for MT_357
Finished inferences for MT_358
Finished inferences for MT_359
Finished inferences for MT_360
Finished inferences for MT_361
Finished inferences for MT_362
Finished inferences for MT_363
Finished inferences for MT_364
Finished inferences for MT_365
Finished inferences for MT_366
Finished inferences for MT_367
Finished inferences for MT_368
Finished inferences for MT_369
Finished inferences for MT_370
Persisted global inferences to /home/de

Finished inferences for MT_227
Finished inferences for MT_228
Finished inferences for MT_229
Finished inferences for MT_230
Finished inferences for MT_231
Finished inferences for MT_232
Finished inferences for MT_233
Finished inferences for MT_234
Finished inferences for MT_235
Finished inferences for MT_236
Finished inferences for MT_237
Finished inferences for MT_238
Finished inferences for MT_239
Finished inferences for MT_240
Finished inferences for MT_241
Finished inferences for MT_242
Finished inferences for MT_243
Finished inferences for MT_244
Finished inferences for MT_245
Finished inferences for MT_246
Finished inferences for MT_247
Finished inferences for MT_248
Finished inferences for MT_249
Finished inferences for MT_250
Finished inferences for MT_251
Finished inferences for MT_252
Finished inferences for MT_253
Finished inferences for MT_254
Finished inferences for MT_255
Finished inferences for MT_256
Finished inferences for MT_257
Finished inferences for MT_258
Finished

Finished inferences for MT_114
Finished inferences for MT_115
Finished inferences for MT_116
Finished inferences for MT_117
Finished inferences for MT_118
Finished inferences for MT_119
Finished inferences for MT_120
Finished inferences for MT_121
Finished inferences for MT_122
Finished inferences for MT_123
Finished inferences for MT_124
Finished inferences for MT_125
Finished inferences for MT_126
Finished inferences for MT_127
Finished inferences for MT_128
Finished inferences for MT_129
Finished inferences for MT_130
Finished inferences for MT_131
Finished inferences for MT_132
Finished inferences for MT_133
Finished inferences for MT_134
Finished inferences for MT_135
Finished inferences for MT_136
Finished inferences for MT_137
Finished inferences for MT_138
Finished inferences for MT_139
Finished inferences for MT_140
Finished inferences for MT_141
Finished inferences for MT_142
Finished inferences for MT_143
Finished inferences for MT_144
Finished inferences for MT_145
Finished

Finished inferences for MT_001
Finished inferences for MT_002
Finished inferences for MT_003
Finished inferences for MT_004
Finished inferences for MT_005
Finished inferences for MT_006
Finished inferences for MT_007
Finished inferences for MT_008
Finished inferences for MT_009
Finished inferences for MT_010
Finished inferences for MT_011
Finished inferences for MT_012
Finished inferences for MT_013
Finished inferences for MT_014
Finished inferences for MT_015
Finished inferences for MT_016
Finished inferences for MT_017
Finished inferences for MT_018
Finished inferences for MT_019
Finished inferences for MT_020
Finished inferences for MT_021
Finished inferences for MT_022
Finished inferences for MT_023
Finished inferences for MT_024
Finished inferences for MT_025
Finished inferences for MT_026
Finished inferences for MT_027
Finished inferences for MT_028
Finished inferences for MT_029
Finished inferences for MT_030
Finished inferences for MT_031
Finished inferences for MT_032
Finished

Finished inferences for MT_266
Finished inferences for MT_267
Finished inferences for MT_268
Finished inferences for MT_269
Finished inferences for MT_270
Finished inferences for MT_271
Finished inferences for MT_272
Finished inferences for MT_273
Finished inferences for MT_274
Finished inferences for MT_275
Finished inferences for MT_276
Finished inferences for MT_277
Finished inferences for MT_278
Finished inferences for MT_279
Finished inferences for MT_280
Finished inferences for MT_281
Finished inferences for MT_282
Finished inferences for MT_283
Finished inferences for MT_284
Finished inferences for MT_285
Finished inferences for MT_286
Finished inferences for MT_287
Finished inferences for MT_288
Finished inferences for MT_289
Finished inferences for MT_290
Finished inferences for MT_291
Finished inferences for MT_292
Finished inferences for MT_293
Finished inferences for MT_294
Finished inferences for MT_295
Finished inferences for MT_296
Finished inferences for MT_297
Finished